# Part(1): Классические модели

## 1. ETS: Разложение прогноза на компоненты

На семинаре мы получили разложение на компоненты на train части. 

Теперь необходимо получить такое разложение для прогноза

$\hat{y}_{t+h} =  l_t + hb_t + s_{t+h-m}$

$l_t = l_{t-1} + b_{t-1}$ 

$b_t = b_{t-1}$

$s_t = s_{t-m}$

#### Обучим модель

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsforecast.utils import AirPassengersDF

HORIZON = 12

df = AirPassengersDF.copy()
data = df["y"].values

model = ExponentialSmoothing(data, seasonal_periods=12, trend="add", seasonal="add", initialization_method="estimated")
result = model.fit()

#### Вынем нужные компоненты

In [ ]:
level_forecast = ...  # <Your code here>
slope_forecast = ...  # <Your code here>
seasonal_forecast = ...  # <Your code here>

#### Проверим результат

In [ ]:
forecast = result.forecast(HORIZON)
assert np.all(forecast == level_forecast + slope_forecast + seasonal_forecast)

## 2. Кроссвалидация в Prophet

На семинаре мы научились валидировать модели из Statsforecast, а что насчет модели Prophet?

Настройте пайплайн валидации и для него!

#### Загрузка данных

In [ ]:
from datasetsforecast.m4 import M4
from statsforecast import StatsForecast
import pandas as pd
import numpy as np

np.random.seed(42)


HORIZON = 24 * 7
N_SEGMENTS = 10

df, *_ = M4.load(directory="data", group="Hourly")

sample_seg = np.random.choice(
    df["unique_id"].unique(),
    size=N_SEGMENTS,
    replace=False,
)
df_sample = df[df["unique_id"].isin(sample_seg)]

for seg in df_sample["unique_id"].unique():
    timestamp = pd.date_range(
        end="2025-01-01", freq="h", periods=df_sample[df_sample["unique_id"] == seg]["ds"].max() + 1
    )
    timestamp_mapping = {i + 1: time for i, time in enumerate(timestamp)}
    df_sample.loc[df_sample["unique_id"] == seg, "ds"] = df_sample["ds"].map(timestamp_mapping)

StatsForecast.plot(df_sample)

#### Кросс-валидация(1): получение исторических прогнозов

Для каждого ряда получите исторические прогнозы

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation

cutoffs = [
    pd.Timestamp("2025-01-01") - pd.Timedelta(15, unit="D"),
    pd.Timestamp("2025-01-01") - pd.Timedelta(8, unit="D"),
]

historical_forecasts = {}

# <Your code here>

#### Кросс-валидация(2): рассчет метрик

Для каждого ряда посчитайте среднюю метрику качества(обратите внимание на параметр `rolling_window`)


In [ ]:
from prophet.diagnostics import performance_metrics

metrics = []

# <Your code here>

metrics = pd.concat(metrics)
metrics

#### Кросс-валидация(3): визуализация

Визуализируйте "реальные данные" VS "исторические предсказания" для ряда с самым плохим mape

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

BAD_SEGMENT = "???"

df_segment = df_sample[df_sample["unique_id"] == "H162"]
df_cv_segment = historical_forecasts[BAD_SEGMENT]

# <Your code here>